In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
y_train = train_df.target
X_train = train_df.drop(columns=["target"])

In [4]:
import re
def remove_digits(DF):
    for i in np.arange(len(DF)):
        DF[i] = re.sub(r"\d+", "", DF[i], flags=re.UNICODE)

In [5]:
X_train = X_train.title
remove_digits(X_train)

In [6]:
test_df.head(10)

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67
5,135314,ryazan.gorodrabot.ru,"Вакансия Мерчендайзер визитный в Рязани, Групп..."
6,135315,m.fotosklad.ru,Смартфон Apple iPhone XR 64GB Черный A2105 EU ...
7,135316,sim-dealer.ru,Духовой шкаф Siemens HB337GYS0R - Sim-Dealer. ...
8,135317,krasnoyarsk.hh.ru,Вакансия Аналитик Департамента внедрения инфор...
9,135318,vsetop.org,Anomaly Defenders v1.0 – торрент


In [7]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english') + stopwords.words('russian'))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words=stop_words)

In [9]:
X_train_vectorized = vectorizer.fit_transform(X_train)
feature_names = np.array(vectorizer.get_feature_names())
for i in np.arange(len(X_train)):
    x_vector = X_train_vectorized.getrow(i).toarray()[0]
    X_train[i] = [feature for feature in feature_names[x_vector > 0]]

In [11]:
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec
size = 100
print(cpu_count())

8


In [12]:
w2v_model = Word2Vec(X_train, min_count = 1, workers = cpu_count(), size=size) 
w2v_model.train(X_train, total_examples = w2v_model.corpus_count, epochs = 3) 

(2950760, 3082431)

In [13]:
w2v_model.wv.most_similar('boobs')

[('ass', 0.9973467588424683),
 ('brunette', 0.9970313310623169),
 ('big', 0.9966017007827759),
 ('amateur', 0.9961438179016113),
 ('blonde', 0.9956956505775452),
 ('cock', 0.9942702651023865),
 ('babe', 0.9938454627990723),
 ('busty', 0.9934890270233154),
 ('ann', 0.9932354092597961),
 ('cum', 0.9922640323638916)]

In [14]:
def convert(data):
    X = np.zeros((len(data), size))
    for i in np.arange(len(data)):
        for word in data[i]:
            length = len(data[i])
            X[i] += w2v_model.wv[word]/length
    return X
X = convert(X_train)
print(X)

[[-0.37745202  0.46288394 -0.12085129 ...  0.22849409  0.11009969
   0.18231687]
 [-0.44141312  0.41080603  0.03860883 ...  0.17092167  0.01580786
   0.12276985]
 [-2.10621036  1.4406051   0.44077935 ...  0.31721446 -0.18933716
  -0.0037977 ]
 ...
 [-0.42087844  0.50586717  0.06154004 ...  0.79018445 -0.33405273
   0.02449279]
 [-0.34510349 -0.03569245  0.71730133 ...  0.99335116  0.09556121
   0.16536962]
 [-0.35492505 -0.22959937  0.93018913 ...  1.24929148  0.36449476
   0.29320618]]


In [15]:
X_test = test_df.drop(columns=[])
X_test = X_test.title
remove_digits(X_test)
vectorize(X_test, vectorizer)

In [16]:
from nltk.cluster.util import cosine_distance

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(metric=cosine_distance, n_neighbors=3, n_jobs=-1)
knn.fit(X, y_train)

C:\Anaconda\lib\site-packages\nltk\cluster\util.py:133: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


KNeighborsClassifier(algorithm='auto', leaf_size=30,
                     metric=<function cosine_distance at 0x000001FBBFE8C798>,
                     metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
                     weights='uniform')

In [18]:
Y = convert(X_test)

In [19]:
y_pred = knn.predict(Y)

C:\Anaconda\lib\site-packages\nltk\cluster\util.py:133: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


In [20]:
test_df["target"] = y_pred.astype(bool)
test_df[["id", "target"]].to_csv("Smirnov02.csv", index=False)

In [24]:
print(y_pred)

[False False  True ... False False False]
